In [2]:
import pandas as pd
import ray
from multiprocessing import Process, Pipe
import xgboost as xgb
import numpy as np 
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier, GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline
import tensorflow
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score
import matplotlib.gridspec as gridspec
from kpal.klib import Profile
from multiprocessing import Queue
from multiprocessing import Process
from Bio.Seq import Seq
from Bio import SeqIO
from multiprocessing import Pool, freeze_support
from scipy.stats import fisher_exact
import subprocess
import re
from statsmodels.stats.multitest import fdrcorrection
import seaborn as sns

/home/pitikovegor/anaconda/anaconda/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/pitikovegor/anaconda/anaconda/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/home/pitikovegor/anaconda/anaconda/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
/home/pitikovegor/anaconda/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future ve

In [3]:
chr_names=['chrX', 'chrY']
for i in range(1, 23):
    chr_names.append('chr%d' % (i))

In [4]:
cell_type = 'K562'
experiment = 'GSM4041593_K562_rep1.hg38'
cont_data_path = '/home/pitikovegor/CourseWork/genomes/reds_experiments/%s/parsed_contacts_%s' % (cell_type, experiment)
sequence_path = '/home/pitikovegor/CourseWork/genomes/chr_sequence'
k = 4

In [7]:
a = '/home/pitikovegor/CourseWork/genomes/reds_experiments/K562/parsed_contacts_GSM4041593_K562_rep1.hg38/predict_chr10_xgb_clusters_10:24:55_4_try4_250_1.tsv'
#/home/pitikovegor/CourseWork/genomes/reds_experiments/K562/parsed_contacts_GSM4041593_K562_rep1.hg38/predict_xgb_trans_07:22:39_4_GSM4041595_K562_rep2.hg38_250_1.tsv

b = '/home/pitikovegor/CourseWork/genomes/reds_experiments/K562/parsed_contacts_GSM4041595_K562_rep2.hg38/predict_chr10_xgb_clusters_10:24:55_4_try4_250_1.tsv'
#/home/pitikovegor/CourseWork/genomes/reds_experiments/K562/parsed_contacts_GSM4041595_K562_rep2.hg38/predict_chr10_xgb_trans_07:22:39_4_GSM4041595_K562_rep2.hg38_250_1.tsv 

In [8]:
c = pd.read_csv(a, sep = '\t')
b = pd.read_csv(b, sep = '\t')

In [9]:
c

,M01460:89:000000000-AJJ2J:1:1101:8549:1192,0.9903077483177185,0.009692219085991383
0,M01460:89:000000000-AJJ2J:1:1101:20437:1195,0.991549,0.008450
1,M01460:89:000000000-AJJ2J:1:1101:17201:1247,0.859015,0.140985
2,M01460:89:000000000-AJJ2J:1:1101:9293:1264,0.997455,0.002545
3,M01460:89:000000000-AJJ2J:1:1101:13743:1285,0.540319,0.459681
4,M01460:89:000000000-AJJ2J:1:1101:17046:1320,0.991053,0.008947
...,...,...,...
58901,M01460:89:000000000-AJJ2J:1:2119:12282:24964,0.977416,0.022584
58902,M01460:89:000000000-AJJ2J:1:2119:20543:25155,0.919363,0.080637
58903,M01460:89:000000000-AJJ2J:1:2119:17368:25184,0.998325,0.001675
58904,M01460:89:000000000-AJJ2J:1:2119:13159:25184,0.987823,0.012177


In [10]:
b

,D00795:30:CA2UTANXX:1:1107:1842:1950,0.9883590936660767,0.011640873737633228
0,D00795:30:CA2UTANXX:1:1107:2260:1890,0.979789,0.020211
1,D00795:30:CA2UTANXX:1:1107:2308:1969,0.952875,0.047125
2,D00795:30:CA2UTANXX:1:1107:2711:1879,0.624905,0.375095
3,D00795:30:CA2UTANXX:1:1107:3941:1925,0.957007,0.042993
4,D00795:30:CA2UTANXX:1:1107:9065:1966,0.975515,0.024485
...,...,...,...
872450,D00795:30:CA2UTANXX:5:2313:15687:4538,0.697368,0.302632
872451,D00795:30:CA2UTANXX:5:2313:16164:4542,0.959225,0.040775
872452,D00795:30:CA2UTANXX:5:2313:20857:4533,0.967326,0.032675
872453,D00795:30:CA2UTANXX:5:2313:1425:4906,0.985078,0.014922
